# SOLID Principles - Interview Preparation

This notebook covers the five SOLID principles of object-oriented design with practical Python examples.

## Table of Contents
1. [Single Responsibility Principle (SRP)](#1-single-responsibility-principle-srp)
2. [Open/Closed Principle (OCP)](#2-openclosed-principle-ocp)
3. [Liskov Substitution Principle (LSP)](#3-liskov-substitution-principle-lsp)
4. [Interface Segregation Principle (ISP)](#4-interface-segregation-principle-isp)
5. [Dependency Inversion Principle (DIP)](#5-dependency-inversion-principle-dip)
6. [Integration Example](#6-integration-example---all-solid-principles)
7. [Interview Q&A](#7-interview-qa)

## Overview

**SOLID** is an acronym for five design principles that make software designs more understandable, flexible, and maintainable:

- **S**ingle Responsibility Principle
- **O**pen/Closed Principle
- **L**iskov Substitution Principle
- **I**nterface Segregation Principle
- **D**ependency Inversion Principle

## 1. Single Responsibility Principle (SRP)

**Definition**: A class should have only one reason to change, meaning it should have only one job or responsibility.

### ❌ Bad Example: Multiple Responsibilities

In [ ]:
# BAD: User class doing too much
import re
import json

class User:
    def __init__(self, username, email):
        self.username = username
        self.email = email
    
    # Responsibility 1: Validation
    def validate_email(self):
        pattern = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
        return re.match(pattern, self.email) is not None
    
    # Responsibility 2: Database operations
    def save_to_database(self):
        # Simulating database save
        print(f"Saving {self.username} to database...")
        return True
    
    # Responsibility 3: Email notification
    def send_welcome_email(self):
        print(f"Sending welcome email to {self.email}...")
        return True
    
    # Responsibility 4: Reporting
    def generate_report(self):
        return f"User Report: {self.username} ({self.email})"

# This class has 4 reasons to change:
# 1. Email validation rules change
# 2. Database schema changes
# 3. Email service provider changes
# 4. Report format changes

user = User("john_doe", "john@example.com")
print(f"Email valid: {user.validate_email()}")
user.save_to_database()
user.send_welcome_email()
print(user.generate_report())

### ✅ Good Example: Single Responsibility

In [ ]:
# GOOD: Each class has a single responsibility
import re

class User:
    """Responsibility: Represent user data"""
    def __init__(self, username, email):
        self.username = username
        self.email = email

class EmailValidator:
    """Responsibility: Validate email addresses"""
    @staticmethod
    def is_valid(email):
        pattern = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
        return re.match(pattern, email) is not None

class UserRepository:
    """Responsibility: Handle user data persistence"""
    def save(self, user):
        print(f"Saving {user.username} to database...")
        return True
    
    def find_by_username(self, username):
        print(f"Finding user: {username}")
        return None

class EmailService:
    """Responsibility: Send emails"""
    def send_welcome_email(self, user):
        print(f"Sending welcome email to {user.email}...")
        return True

class UserReportGenerator:
    """Responsibility: Generate user reports"""
    def generate(self, user):
        return f"User Report: {user.username} ({user.email})"

# Usage
user = User("john_doe", "john@example.com")

validator = EmailValidator()
print(f"Email valid: {validator.is_valid(user.email)}")

repository = UserRepository()
repository.save(user)

email_service = EmailService()
email_service.send_welcome_email(user)

report_gen = UserReportGenerator()
print(report_gen.generate(user))

print("\n✅ Each class now has a single, well-defined responsibility!")

## 2. Open/Closed Principle (OCP)

**Definition**: Software entities should be open for extension but closed for modification.

### ❌ Bad Example: Modifying Existing Code

In [ ]:
# BAD: Need to modify class every time we add a payment method
class PaymentProcessor:
    def process_payment(self, payment_type, amount):
        if payment_type == "credit_card":
            print(f"Processing ${amount} via Credit Card")
            print("Validating card number...")
            print("Charging card...")
        elif payment_type == "paypal":
            print(f"Processing ${amount} via PayPal")
            print("Redirecting to PayPal...")
            print("Confirming payment...")
        elif payment_type == "bitcoin":
            # Every new payment type requires modifying this class!
            print(f"Processing ${amount} via Bitcoin")
            print("Generating wallet address...")
            print("Waiting for confirmations...")
        else:
            raise ValueError(f"Unknown payment type: {payment_type}")

processor = PaymentProcessor()
processor.process_payment("credit_card", 100)
print()
processor.process_payment("paypal", 50)
print()
processor.process_payment("bitcoin", 200)

### ✅ Good Example: Extension Without Modification

In [ ]:
# GOOD: Can add new payment methods without modifying existing code
from abc import ABC, abstractmethod

class PaymentMethod(ABC):
    """Abstract base class for all payment methods"""
    @abstractmethod
    def process(self, amount):
        pass

class CreditCardPayment(PaymentMethod):
    def process(self, amount):
        print(f"Processing ${amount} via Credit Card")
        print("Validating card number...")
        print("Charging card...")

class PayPalPayment(PaymentMethod):
    def process(self, amount):
        print(f"Processing ${amount} via PayPal")
        print("Redirecting to PayPal...")
        print("Confirming payment...")

class BitcoinPayment(PaymentMethod):
    def process(self, amount):
        print(f"Processing ${amount} via Bitcoin")
        print("Generating wallet address...")
        print("Waiting for confirmations...")

# Can easily add new payment method without modifying existing code!
class ApplePayPayment(PaymentMethod):
    def process(self, amount):
        print(f"Processing ${amount} via Apple Pay")
        print("Authenticating with Face ID...")
        print("Completing payment...")

class PaymentProcessor:
    """Processes any payment method - no modification needed for new types!"""
    def process_payment(self, payment_method: PaymentMethod, amount):
        payment_method.process(amount)

# Usage
processor = PaymentProcessor()

processor.process_payment(CreditCardPayment(), 100)
print()
processor.process_payment(PayPalPayment(), 50)
print()
processor.process_payment(BitcoinPayment(), 200)
print()
processor.process_payment(ApplePayPayment(), 75)

print("\n✅ Added Apple Pay without modifying existing code!")

## 3. Liskov Substitution Principle (LSP)

**Definition**: Objects of a superclass should be replaceable with objects of a subclass without breaking the application.

### ❌ Bad Example: Violating LSP

In [ ]:
# BAD: Square violates LSP - can't substitute Rectangle with Square
class Rectangle:
    def __init__(self, width, height):
        self._width = width
        self._height = height
    
    def set_width(self, width):
        self._width = width
    
    def set_height(self, height):
        self._height = height
    
    def area(self):
        return self._width * self._height

class Square(Rectangle):
    def __init__(self, side):
        super().__init__(side, side)
    
    def set_width(self, width):
        # Square must maintain equal sides - violates LSP!
        self._width = width
        self._height = width
    
    def set_height(self, height):
        # Square must maintain equal sides - violates LSP!
        self._width = height
        self._height = height

def test_rectangle(rect: Rectangle):
    """This function expects Rectangle behavior"""
    rect.set_width(5)
    rect.set_height(4)
    expected_area = 5 * 4  # 20
    actual_area = rect.area()
    print(f"Expected area: {expected_area}, Actual area: {actual_area}")
    assert actual_area == expected_area, "LSP Violated!"

# Works fine with Rectangle
rect = Rectangle(0, 0)
test_rectangle(rect)
print("✅ Rectangle works as expected\n")

# Breaks with Square (LSP violation)
square = Square(0)
try:
    test_rectangle(square)
except AssertionError as e:
    print(f"❌ {e}")
    print("Square cannot be substituted for Rectangle - LSP violated!")

### ✅ Good Example: Following LSP

In [ ]:
# GOOD: Separate hierarchies that respect LSP
from abc import ABC, abstractmethod

class Shape(ABC):
    """Base class for all shapes"""
    @abstractmethod
    def area(self):
        pass

class Rectangle(Shape):
    def __init__(self, width, height):
        self._width = width
        self._height = height
    
    def set_width(self, width):
        self._width = width
    
    def set_height(self, height):
        self._height = height
    
    def area(self):
        return self._width * self._height

class Square(Shape):
    """Square is NOT a Rectangle - it's its own shape"""
    def __init__(self, side):
        self._side = side
    
    def set_side(self, side):
        self._side = side
    
    def area(self):
        return self._side * self._side

# Any Shape can be used interchangeably
def print_area(shape: Shape):
    print(f"Area: {shape.area()}")

rect = Rectangle(5, 4)
square = Square(5)

print_area(rect)   # Works
print_area(square)  # Works

print("\n✅ Both Rectangle and Square can substitute Shape without issues!")

## 4. Interface Segregation Principle (ISP)

**Definition**: A client should not be forced to depend on methods it does not use.

### ❌ Bad Example: Fat Interface

In [ ]:
# BAD: One large interface forces classes to implement methods they don't need
from abc import ABC, abstractmethod

class Worker(ABC):
    """Fat interface - not all workers can do all these things!"""
    @abstractmethod
    def work(self):
        pass
    
    @abstractmethod
    def eat(self):
        pass
    
    @abstractmethod
    def sleep(self):
        pass

class HumanWorker(Worker):
    def work(self):
        print("Human working...")
    
    def eat(self):
        print("Human eating...")
    
    def sleep(self):
        print("Human sleeping...")

class RobotWorker(Worker):
    def work(self):
        print("Robot working...")
    
    def eat(self):
        # Robots don't eat! Forced to implement unused method
        raise NotImplementedError("Robots don't eat!")
    
    def sleep(self):
        # Robots don't sleep! Forced to implement unused method
        raise NotImplementedError("Robots don't sleep!")

# Test
human = HumanWorker()
human.work()
human.eat()
human.sleep()

print()
robot = RobotWorker()
robot.work()
try:
    robot.eat()
except NotImplementedError as e:
    print(f"❌ {e}")

print("\n❌ Robot forced to implement methods it doesn't need!")

### ✅ Good Example: Segregated Interfaces

In [ ]:
# GOOD: Split into smaller, focused interfaces
from abc import ABC, abstractmethod

class Workable(ABC):
    """Interface for anything that can work"""
    @abstractmethod
    def work(self):
        pass

class Eatable(ABC):
    """Interface for anything that can eat"""
    @abstractmethod
    def eat(self):
        pass

class Sleepable(ABC):
    """Interface for anything that can sleep"""
    @abstractmethod
    def sleep(self):
        pass

class HumanWorker(Workable, Eatable, Sleepable):
    """Humans implement all three interfaces"""
    def work(self):
        print("Human working...")
    
    def eat(self):
        print("Human eating...")
    
    def sleep(self):
        print("Human sleeping...")

class RobotWorker(Workable):
    """Robots only implement Workable - no unused methods!"""
    def work(self):
        print("Robot working...")
    
    def charge(self):
        print("Robot charging...")

# Test
def manage_worker(worker: Workable):
    worker.work()

def manage_break(worker: Eatable):
    worker.eat()

human = HumanWorker()
robot = RobotWorker()

manage_worker(human)  # Works
manage_worker(robot)  # Works

print()
manage_break(human)   # Works
# manage_break(robot) # Would cause compile error - robot doesn't implement Eatable

print()
robot.charge()

print("\n✅ Each class only implements the interfaces it needs!")

## 5. Dependency Inversion Principle (DIP)

**Definition**: High-level modules should not depend on low-level modules. Both should depend on abstractions.

### ❌ Bad Example: High-level depends on Low-level

In [ ]:
# BAD: UserService directly depends on concrete MySQLDatabase
class MySQLDatabase:
    """Low-level module - concrete implementation"""
    def connect(self):
        print("Connecting to MySQL...")
    
    def save_user(self, user):
        print(f"Saving {user} to MySQL database")

class UserService:
    """High-level module depends on low-level MySQLDatabase"""
    def __init__(self):
        # Tightly coupled to MySQL - can't easily switch databases!
        self.database = MySQLDatabase()
    
    def create_user(self, username):
        print(f"Creating user: {username}")
        self.database.connect()
        self.database.save_user(username)

# Usage
service = UserService()
service.create_user("john_doe")

print("\n❌ UserService is tightly coupled to MySQL!")
print("   Switching to PostgreSQL would require modifying UserService.")

### ✅ Good Example: Both depend on Abstractions

In [ ]:
# GOOD: Both high-level and low-level depend on abstraction
from abc import ABC, abstractmethod

class Database(ABC):
    """Abstraction - both high and low level depend on this"""
    @abstractmethod
    def connect(self):
        pass
    
    @abstractmethod
    def save_user(self, user):
        pass

class MySQLDatabase(Database):
    """Low-level module - implements abstraction"""
    def connect(self):
        print("Connecting to MySQL...")
    
    def save_user(self, user):
        print(f"Saving {user} to MySQL database")

class PostgreSQLDatabase(Database):
    """Low-level module - implements abstraction"""
    def connect(self):
        print("Connecting to PostgreSQL...")
    
    def save_user(self, user):
        print(f"Saving {user} to PostgreSQL database")

class MongoDBDatabase(Database):
    """Low-level module - implements abstraction"""
    def connect(self):
        print("Connecting to MongoDB...")
    
    def save_user(self, user):
        print(f"Saving {user} to MongoDB database")

class UserService:
    """High-level module - depends on abstraction, not concrete implementation"""
    def __init__(self, database: Database):
        # Dependency is injected - loosely coupled!
        self.database = database
    
    def create_user(self, username):
        print(f"Creating user: {username}")
        self.database.connect()
        self.database.save_user(username)

# Usage - can easily switch database implementations!
print("Using MySQL:")
mysql_service = UserService(MySQLDatabase())
mysql_service.create_user("john_doe")

print("\nUsing PostgreSQL:")
postgres_service = UserService(PostgreSQLDatabase())
postgres_service.create_user("jane_doe")

print("\nUsing MongoDB:")
mongo_service = UserService(MongoDBDatabase())
mongo_service.create_user("bob_smith")

print("\n✅ UserService is decoupled from specific database implementations!")
print("   Can switch databases without modifying UserService.")

## 6. Integration Example - All SOLID Principles

Let's build a comprehensive example that demonstrates all five SOLID principles working together.

In [ ]:
from abc import ABC, abstractmethod
from typing import List

# ============================================================================
# INTERFACES (ISP - Interface Segregation Principle)
# ============================================================================

class Notifiable(ABC):
    """Interface for sending notifications"""
    @abstractmethod
    def send(self, recipient: str, message: str) -> bool:
        pass

class Persistable(ABC):
    """Interface for data persistence"""
    @abstractmethod
    def save(self, data: dict) -> bool:
        pass
    
    @abstractmethod
    def find(self, id: str) -> dict:
        pass

# ============================================================================
# LOW-LEVEL MODULES (DIP - Dependency Inversion + OCP - Open/Closed)
# ============================================================================

class EmailNotification(Notifiable):
    """Can extend with new notification types without modifying existing code"""
    def send(self, recipient: str, message: str) -> bool:
        print(f"📧 Email sent to {recipient}: {message}")
        return True

class SMSNotification(Notifiable):
    """Another notification type - OCP in action"""
    def send(self, recipient: str, message: str) -> bool:
        print(f"📱 SMS sent to {recipient}: {message}")
        return True

class DatabaseRepository(Persistable):
    """Database storage implementation"""
    def save(self, data: dict) -> bool:
        print(f"💾 Saved to database: {data}")
        return True
    
    def find(self, id: str) -> dict:
        print(f"🔍 Finding order {id} in database")
        return {"id": id, "status": "found"}

class CacheRepository(Persistable):
    """Cache storage implementation - can swap without breaking code"""
    def save(self, data: dict) -> bool:
        print(f"⚡ Saved to cache: {data}")
        return True
    
    def find(self, id: str) -> dict:
        print(f"⚡ Finding order {id} in cache")
        return {"id": id, "status": "found"}

# ============================================================================
# DOMAIN ENTITIES (SRP - Single Responsibility)
# ============================================================================

class Order:
    """SRP: Only responsible for representing order data"""
    def __init__(self, order_id: str, customer_email: str, amount: float):
        self.order_id = order_id
        self.customer_email = customer_email
        self.amount = amount
        self.status = "pending"
    
    def to_dict(self) -> dict:
        return {
            "order_id": self.order_id,
            "customer_email": self.customer_email,
            "amount": self.amount,
            "status": self.status
        }

# ============================================================================
# SERVICE CLASSES (SRP - Each has single responsibility)
# ============================================================================

class OrderValidator:
    """SRP: Only responsible for order validation"""
    def validate(self, order: Order) -> bool:
        if not order.order_id:
            print("❌ Invalid order: missing ID")
            return False
        if order.amount <= 0:
            print("❌ Invalid order: amount must be positive")
            return False
        print("✅ Order validation passed")
        return True

class NotificationService:
    """SRP: Only responsible for coordinating notifications"""
    """DIP: Depends on Notifiable abstraction, not concrete implementations"""
    def __init__(self, notifiers: List[Notifiable]):
        self.notifiers = notifiers
    
    def notify(self, recipient: str, message: str):
        for notifier in self.notifiers:
            notifier.send(recipient, message)

class OrderService:
    """High-level business logic"""
    """DIP: Depends on abstractions (Persistable, NotificationService)"""
    """SRP: Only responsible for order processing workflow"""
    def __init__(
        self,
        repository: Persistable,
        notification_service: NotificationService,
        validator: OrderValidator
    ):
        self.repository = repository
        self.notification_service = notification_service
        self.validator = validator
    
    def process_order(self, order: Order) -> bool:
        print(f"\n{'='*60}")
        print(f"Processing Order: {order.order_id}")
        print(f"{'='*60}")
        
        # Validate
        if not self.validator.validate(order):
            return False
        
        # Save
        order.status = "confirmed"
        self.repository.save(order.to_dict())
        
        # Notify
        message = f"Order {order.order_id} confirmed! Amount: ${order.amount}"
        self.notification_service.notify(order.customer_email, message)
        
        print(f"✅ Order {order.order_id} processed successfully!")
        return True

# ============================================================================
# USAGE - Demonstrating LSP (Liskov Substitution Principle)
# ============================================================================

print("\n" + "="*60)
print("SOLID PRINCIPLES INTEGRATION DEMO")
print("="*60)

# Configuration 1: Database storage + Email notification
print("\n📦 Configuration 1: Database + Email")
order_service1 = OrderService(
    repository=DatabaseRepository(),
    notification_service=NotificationService([EmailNotification()]),
    validator=OrderValidator()
)

order1 = Order("ORD-001", "customer1@example.com", 99.99)
order_service1.process_order(order1)

# Configuration 2: Cache storage + Email + SMS notifications
# LSP: Can substitute DatabaseRepository with CacheRepository
# OCP: Added SMS notification without modifying existing code
print("\n📦 Configuration 2: Cache + Email + SMS")
order_service2 = OrderService(
    repository=CacheRepository(),  # LSP: Substitution works!
    notification_service=NotificationService([
        EmailNotification(),
        SMSNotification()  # OCP: Extension without modification!
    ]),
    validator=OrderValidator()
)

order2 = Order("ORD-002", "customer2@example.com", 149.99)
order_service2.process_order(order2)

print("\n" + "="*60)
print("SOLID PRINCIPLES DEMONSTRATED:")
print("="*60)
print("✅ SRP: Each class has a single, well-defined responsibility")
print("✅ OCP: Added SMS notification without modifying existing code")
print("✅ LSP: Swapped DatabaseRepository with CacheRepository seamlessly")
print("✅ ISP: Separate Notifiable and Persistable interfaces")
print("✅ DIP: High-level OrderService depends on abstractions")
print("="*60)

## 7. Interview Q&A

### Q1: What are the SOLID principles?

**Answer**: SOLID is an acronym for five object-oriented design principles:
- **S**ingle Responsibility: A class should have only one reason to change
- **O**pen/Closed: Open for extension, closed for modification
- **L**iskov Substitution: Subtypes must be substitutable for their base types
- **I**nterface Segregation: Clients shouldn't depend on unused methods
- **D**ependency Inversion: Depend on abstractions, not concretions

### Q2: Why is the Single Responsibility Principle important?

**Answer**: SRP makes code more maintainable because:
- Changes to one responsibility don't affect others
- Easier to understand (each class has one clear purpose)
- Easier to test (fewer dependencies)
- Reduces coupling between different parts of the system

### Q3: How does the Open/Closed Principle reduce bugs?

**Answer**: OCP reduces bugs because:
- Existing, tested code isn't modified when adding features
- New functionality is added through extension (new classes)
- Reduces risk of breaking existing behavior
- Makes code more stable and predictable

### Q4: What's the classic violation of Liskov Substitution Principle?

**Answer**: The Rectangle-Square problem:
- A Square inheriting from Rectangle seems logical mathematically
- But Square must override width/height setters to maintain equal sides
- This breaks the Rectangle's contract that width and height are independent
- Functions expecting Rectangle behavior will fail with Square
- Solution: Make both inherit from a common Shape interface

### Q5: How do you know when to split an interface (ISP)?

**Answer**: Split interfaces when:
- Implementing classes have to stub out or raise NotImplementedError
- Different clients need different subsets of methods
- The interface is growing with unrelated methods
- You find yourself writing empty implementations

### Q6: What's the difference between Dependency Inversion and Dependency Injection?

**Answer**:
- **Dependency Inversion** (DIP): A principle stating that high-level modules should depend on abstractions, not concrete implementations
- **Dependency Injection** (DI): A technique/pattern for implementing DIP where dependencies are provided (injected) from outside rather than created internally
- DI is one way to achieve DIP

### Q7: How do SOLID principles relate to design patterns?

**Answer**: Design patterns often implement SOLID principles:
- **Strategy Pattern**: OCP (add strategies without modification), DIP (depend on strategy interface)
- **Factory Pattern**: OCP (add new product types), DIP (clients depend on product interface)
- **Decorator Pattern**: OCP (add behavior through wrapping), SRP (each decorator has one responsibility)
- **Observer Pattern**: OCP (add observers without modifying subject), ISP (separate observer interfaces)

### Q8: Can you have too much abstraction following SOLID?

**Answer**: Yes, over-engineering is possible:
- Don't create abstractions for code that will never change
- Balance SOLID principles with YAGNI (You Aren't Gonna Need It)
- Start simple, refactor to SOLID when you identify change points
- Consider the actual likelihood of change
- Too many layers can make code harder to understand

### Q9: Which SOLID principle is most often violated?

**Answer**: Single Responsibility Principle:
- Easiest to violate because responsibilities can be vague
- Classes naturally accumulate responsibilities over time
- "God objects" that do everything are common in legacy code
- Requires discipline to keep classes focused

### Q10: How do you refactor code to follow SOLID principles?

**Answer**: Step-by-step approach:
1. **Identify violations**: Look for code smells (large classes, if/else chains, etc.)
2. **Write tests**: Ensure existing behavior before refactoring
3. **Extract responsibilities**: Split classes/methods with multiple responsibilities (SRP)
4. **Introduce abstractions**: Create interfaces/base classes (DIP, OCP)
5. **Use dependency injection**: Pass dependencies rather than creating them (DIP)
6. **Split interfaces**: Break fat interfaces into focused ones (ISP)
7. **Verify substitutability**: Ensure subtypes don't break base type contracts (LSP)
8. **Test again**: Ensure refactoring didn't break functionality

## Summary

### SOLID Principles Quick Reference

| Principle | Key Question | Violation Sign | Quick Fix |
|-----------|-------------|----------------|----------|
| **SRP** | Does this class have only one reason to change? | Class has multiple unrelated methods | Extract classes |
| **OCP** | Can I add features without modifying existing code? | if/elif chains for types | Use polymorphism |
| **LSP** | Can I substitute subclass for superclass? | Subclass breaks parent's contract | Rethink hierarchy |
| **ISP** | Are all interface methods used by implementers? | Empty/NotImplemented methods | Split interface |
| **DIP** | Do I depend on abstractions or concretions? | Direct instantiation of concrete classes | Inject dependencies |

### Benefits of SOLID

- **Maintainability**: Easier to modify and extend
- **Testability**: Smaller, focused classes are easier to test
- **Flexibility**: Can swap implementations easily
- **Reusability**: Well-designed components can be reused
- **Understandability**: Clear responsibilities make code easier to understand

### When to Apply SOLID

✅ **Do apply when:**
- Building medium to large applications
- Requirements are likely to change
- Multiple developers will maintain the code
- Code will be reused across projects

⚠️ **Consider carefully when:**
- Building small scripts or prototypes
- Requirements are completely stable
- Performance is critical (abstraction has overhead)
- Team is unfamiliar with OOP patterns

### Practice Tips

1. Start with SRP - it's the foundation
2. Use OCP when you see if/elif type checking
3. Think LSP when designing inheritance hierarchies
4. Apply ISP when interfaces grow large
5. Use DIP to decouple high-level logic from details
6. Refactor gradually - don't over-engineer upfront
7. Write tests to verify SOLID refactorings don't break behavior